# ThoroughBet Simulation


## Load necessary modules

In [1]:
import numpy as np
import pickle

from utils import settings, timestamp, YEAR
from utils.arrayview import ArrayView, TimeseriesView


from prediction.models.preprocessing import Model
from prediction.models.prediction import factornames_trimmed
from prediction.models.parameters import factor_build_end

## Load data

In [2]:
av = ArrayView.from_file(settings.paths.join('brain_final2cut.av.bcolz'))

In [3]:
tsav = {}
sl = 0
while True:
    try:
        tsav[sl] = ArrayView.from_file(settings.paths.join('brain_final2_slice_%s.av.bcolz' % sl))
    except ValueError:
        break
    sl += 1

## Preprocessing

In [4]:
mod = Model(av, oos_start=factor_build_end+YEAR)

In [5]:
high_kurtosis_factors =  ['z64f5be67e', 'z90adc182a', 'z7081bf371', 'z34b808e99', 'z757be272e', 'z5a85cd6a9',
                         'zf991b634a', 'z62651f605', 'zd002b7067', 'z2ef7fedca', 'z6f11029f7', 'z412893062',
                          'z919b9585a', 'z89b0eda37', 'z31780b3f4', 'z6631693d3', 'z0b27f29ad', 'zd7cd94e4c', 
                          'zf5b2aef2a']

In [6]:
price_factors = ['zb392bb74a', 'z6809c316d', 'zd678f0538', 'z027f9f0f5', 'z88e79930c', 'z4a72dc02f',
                 'z1a3573928', 'z7b15df227']

In [7]:
%time factors = mod._preprocess_factors(factornames_trimmed, high_kurtosis_factors = high_kurtosis_factors,\
                                        price_factors = price_factors, verbose=True)

INFO:models:Getting factors from av and rescaling...


. . . . .

INFO:models:Filling in missing values...
INFO:models:Computing each factor as linear combination of all the others...


 . . . . .

INFO:models:Number of missing patterns: 10443


 . . . . . . . . . . 10000


INFO:models:Transforming factors by applying CL-model on their Taylor expansions...


. . . . .CPU times: user 6min 28s, sys: 20.8 s, total: 6min 49s
Wall time: 6min 21s



In [8]:
factors.T.shape

(1742772, 57)

In [9]:
predict_mask = mod.is1|mod.is2|mod.oos
train_mask = mod.is1|mod.is2
train_event_id = av.event_id[train_mask]
predict_event_id = av.event_id[predict_mask]
len(predict_mask), len(train_mask), train_mask.sum(), predict_mask.sum(), len(train_event_id), len(np.unique(train_event_id))

(1742772, 1742772, 125714, 150297, 125714, 12375)

In [10]:
import pandas as pd
pd.set_option('display.max_columns', 60)

In [11]:
col_names = ['f{}'.format(i) for i in range(1,58)]
df = pd.DataFrame(data =factors[:, predict_mask].T , columns = col_names)

In [12]:
df['event_id'] = av.event_id[predict_mask]
df['runner_id'] = av.runner_id[predict_mask]
df['result'] = av.result[predict_mask]
df['is1'] = mod.is1[predict_mask]
df['is2'] = mod.is2[predict_mask]
df['oos'] = mod.oos[predict_mask]
df['time'] =av.start_time[predict_mask]
df['obstacle'] = av.obstacle[predict_mask]
df['going'] = av.going[predict_mask]
df['speed'] = av.speed[predict_mask]
df['distance'] = av.distance[predict_mask]
df['prize'] = av.prize[predict_mask]

df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,...,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,event_id,runner_id,result,is1,is2,oos,time,obstacle,going,speed,distance,prize
0,0.066400,1.812544,0.175912,0.007658,-0.00007,0.380432,0.886927,-0.003214,1.493022,-0.510089,0.174814,0.784029,0.492731,0.021176,1.222551,0.332255,-0.003245,0.083893,0.003949,-0.018036,0.044318,-0.236929,1.061229,-0.056129,-0.000462,-0.000142,-0.065776,0.000007,-0.017906,0.294096,...,2.493133,0.780797,0.111163,0.222586,0.005423,0.023559,0.630792,0.098600,-0.015562,-0.102575,0.537004,-0.010811,0.056470,0.934733,0.419065,-0.003162,-0.003211,0.115685,293661,360456,3,True,True,False,1.443704e+09,F,GD-FM,15.735644,1700.784058,3235.0
1,-0.154935,0.615559,-0.099403,-0.003315,0.00063,0.151092,-0.100909,0.005656,0.220873,0.121961,0.057041,0.432281,0.044878,0.017920,0.414254,0.201616,0.005911,0.005786,0.022141,-0.015606,-0.026866,0.276907,-0.188122,0.044831,0.004163,-0.001194,0.111790,-0.004704,-0.032841,0.036493,...,1.649089,0.002394,-1.172824,0.226226,0.030542,-0.036105,0.356432,-0.247407,-0.015704,0.589225,-0.121583,0.097302,-0.166858,0.225441,0.053834,0.005173,0.005620,0.052669,293661,375590,5,True,True,False,1.443704e+09,F,GD-FM,15.622030,1700.784058,3235.0
2,0.066400,0.443172,0.064974,0.007658,-0.00007,0.203753,-0.100909,0.011806,-0.030797,-0.497101,0.355908,0.236486,-0.301619,0.027384,0.365929,0.153332,0.011789,0.035920,0.008270,0.013054,-0.014764,0.263956,0.298949,0.041541,-0.000462,0.000254,0.168972,0.001457,0.057498,-0.092061,...,0.942577,0.152497,-0.248882,0.104340,-0.036932,0.084404,0.138569,0.098600,0.079891,-0.147373,-0.183630,-0.010811,0.002193,0.481888,0.037694,0.011866,0.011813,0.044670,293661,374610,7,True,True,False,1.443704e+09,F,GD-FM,15.565223,1700.784058,3235.0
3,-0.154935,0.766351,0.157395,-0.025148,-0.00007,0.114861,-0.257157,0.009880,0.299221,0.391816,-0.260937,-0.046712,-0.033713,0.013927,0.551059,-0.300384,0.009861,-0.052231,0.008270,0.011259,-0.004403,-0.122487,-0.350030,0.062382,-0.000462,-0.001858,0.234969,0.001830,-0.017906,-0.127139,...,0.499952,-0.072498,0.688550,-0.007017,0.000300,-0.019645,-0.160828,-0.247407,-0.015562,0.304180,0.289913,-0.010811,0.417492,-0.145241,-0.497020,0.009939,0.009887,-0.263563,293661,373638,1,True,True,False,1.443704e+09,F,GD-FM,15.849259,1700.784058,3235.0
4,0.066400,0.330113,-0.122287,0.007658,-0.00007,0.114861,0.503362,-0.025454,1.263503,-0.443943,0.086009,-0.046712,-0.033713,0.027452,1.253726,0.168296,-0.025506,-0.016331,0.008270,0.013054,-0.038637,0.105709,0.180534,-0.041776,-0.000462,0.000211,-0.039124,0.002172,0.100686,0.030163,...,-0.003480,0.045434,0.591243,0.105751,0.069936,-0.044642,-0.160828,0.098600,0.022905,-0.193690,-0.440603,-0.010811,-0.426567,0.135662,0.444311,-0.025412,-0.025456,0.263005,293661,347906,4,True,True,False,1.443704e+09,F,GD-FM,15.712922,1700.784058,3235.0


In [13]:
#df.to_csv('/home/oleg/thbmodel/racehorse_data2.csv')

In [13]:
ts_mask = av.start_time >= float(timestamp('2015-08-01'))

for k in tsav.keys():
    tsav[k] = tsav[k][predict_mask [ts_mask]]

- групировка по забегам 
- df_f сгрупированы факторы по забегу c вычислением минимальной разницы между сортированными факторами для значение в забеге
- df1 сгрупированы  переменные которые общие для всех участников звбега 

In [14]:
def mean_max_min(data):
    
    n, m = data.shape
    max_min = np.zeros((n,m))
    index  = data.index
    #print index
    for i,j in enumerate(index):
        #print np.setdiff1d(index, i)
        data_ = data.ix[np.setdiff1d(index, j), :].values
        #print data_[:2]
        max_min[i, :] = (data_.max(axis =0) - data_.min(axis =0))/n
        
    return pd.DataFrame(data = max_min, columns = data.columns)

In [17]:
#df_f = df.ix[:,u'f1':u'f57'].groupby(by = df['event_id']).apply(mean_max_min)
for col in df_f.columns:
    df[str('M_'+col)]= df_f[col].values
pd.options.display.max_columns = 140
df.head(10)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,event_id,runner_id,result,is1,is2,oos,time,obstacle,going,speed,distance,prize,M_f1,M_f2,M_f3,M_f4,M_f5,M_f6,M_f7,M_f8,M_f9,M_f10,M_f11,M_f12,M_f13,M_f14,M_f15,M_f16,M_f17,M_f18,M_f19,M_f20,M_f21,M_f22,M_f23,M_f24,M_f25,M_f26,M_f27,M_f28,M_f29,M_f30,M_f31,M_f32,M_f33,M_f34,M_f35,M_f36,M_f37,M_f38,M_f39,M_f40,M_f41,M_f42,M_f43,M_f44,M_f45,M_f46,M_f47,M_f48,M_f49,M_f50,M_f51,M_f52,M_f53,M_f54,M_f55,M_f56,M_f57
0,0.066400,1.812544,0.175912,0.007658,-0.00007,0.380432,0.886927,-0.003214,1.493022,-0.510089,0.174814,0.784029,0.492731,0.021176,1.222551,0.332255,-0.003245,0.083893,0.003949,-0.018036,0.044318,-0.236929,1.061229,-0.056129,-0.000462,-0.000142,-0.065776,0.000007,-0.017906,0.294096,-0.485075,-0.084142,-0.112596,-0.002112,-0.003081,-0.167638,0.074538,0.967736,-0.003212,2.493133,0.780797,0.111163,0.222586,0.005423,0.023559,0.630792,0.098600,-0.015562,-0.102575,0.537004,-0.010811,0.056470,0.934733,0.419065,-0.003162,-0.003211,0.115685,293661,360456,3,True,True,False,1.443704e+09,F,GD-FM,15.735644,1700.784058,3235.0,0.022134,0.260604,0.043252,0.003281,0.00007,0.155211,0.129012,0.003726,0.424278,0.210182,0.097216,0.121474,0.034650,0.009060,0.311063,0.051448,0.003730,0.008922,0.006000,0.002866,0.011607,0.069193,0.110954,0.013933,0.000463,0.000320,0.059297,0.000698,0.013353,0.018471,0.079228,0.080813,0.063538,0.002113,0.003758,0.116451,0.036222,0.212490,0.003726,0.413493,0.099755,0.186137,0.107289,0.011924,0.012905,0.051726,0.037202,0.009559,0.078291,0.089647,0.010811,0.085310,0.137641,0.098123,0.003728,0.003727,0.071853
1,-0.154935,0.615559,-0.099403,-0.003315,0.00063,0.151092,-0.100909,0.005656,0.220873,0.121961,0.057041,0.432281,0.044878,0.017920,0.414254,0.201616,0.005911,0.005786,0.022141,-0.015606,-0.026866,0.276907,-0.188122,0.044831,0.004163,-0.001194,0.111790,-0.004704,-0.032841,0.036493,-0.033482,0.157650,0.033526,0.019013,0.004390,0.030125,-0.090091,0.255435,0.005635,1.649089,0.002394,-1.172824,0.226226,0.030542,-0.036105,0.356432,-0.247407,-0.015704,0.589225,-0.121583,0.097302,-0.166858,0.225441,0.053834,0.005173,0.005620,0.052669,293661,375590,5,True,True,False,1.443704e+09,F,GD-FM,15.622030,1700.784058,3235.0,0.022134,0.365223,0.043252,0.003281,0.00000,0.155211,0.152782,0.003726,0.447230,0.210182,0.097216,0.156649,0.079435,0.009060,0.311063,0.064512,0.003730,0.013719,0.004612,0.003109,0.011630,0.067898,0.187072,0.013933,0.000000,0.000320,0.059297,0.000656,0.011859,0.042123,0.100929,0.080813,0.063538,0.000000,0.003758,0.116451,0.036222,0.212490,0.003726,0.497897,0.146564,0.103327,0.107289,0.011924,0.012905,0.079162,0.037202,0.009545,0.049787,0.097761,0.000000,0.085310,0.182926,0.098123,0.003728,0.003727,0.071853
2,0.066400,0.443172,0.064974,0.007658,-0.00007,0.203753,-0.100909,0.011806,-0.030797,-0.497101,0.355908,0.236486,-0.301619,0.027384,0.365929,0.153332,0.011789,0.035920,0.008270,0.013054,-0.014764,0.263956,0.298949,0.041541,-0.000462,0.000254,0.168972,0.001457,0.057498,-0.092061,-0.268066,-0.060863,0.000601,-0.002112,0.012036,0.175475,-0.025941,0.156538,0.011807,0.942577,0.152497,-0.248882,0.104340,-0.036932,0.084404,0.138569,0.098600,0.079891,-0.147373,-0.183630,-0.010811,0.002193,0.481888,0.037694,0.011866,0.011813,0.044670,293661,374610,7,True,True,False,1.443704e+09,F,GD-FM,15.565223,1700.784058,3235.0,0.022134,0.365223,0.043252,0.003281,0.00007,0.155211,0.152782,0.003533,0.447230,0.210182,0.092662,0.156649,0.052644,0.009060,0.311063,0.064512,0.003537,0.013719,0.006000,0.003109,0.011630,0.069193,0.187072,0.013933,0.000463,0.000320,0.059297,0.000698,0.013353,0.042123,0.100929,0.080813,0.063538,0.002113,0.003565,0.116451,0.036222,0.212490,0.003533,0.497897,0.146564,0.186137,0.107289,0.011924,0.008795,0.079162,0.037202,0.003861,0.078291,0.097761,0.010811,0.085310,0.182926,0.098123,0.003535,0.00

In [18]:
df_f.shape, df.shape

((150297, 57), (150297, 126))

In [19]:
from prediction.models import clmodel
from prediction.tools.helpers import strata_scale_down

is1 = mod.is1.copy()
is2 = mod.is2.copy()
oos = mod.oos.copy()
strata = strata_scale_down(av.event_id)

In [20]:
df_f.head()

f1        f2        f3        f4       f5        f6  \
event_id                                                                
293661   0  0.022134  0.260604  0.043252  0.003281  0.00007  0.155211   
         1  0.022134  0.365223  0.043252  0.003281  0.00000  0.155211   
         2  0.022134  0.365223  0.043252  0.003281  0.00007  0.155211   
         3  0.022134  0.365223  0.043252  0.003281  0.00007  0.155211   
         4  0.022134  0.365223  0.043252  0.003281  0.00007  0.155211   

                  f7        f8        f9       f10       f11       f12  \
event_id                                                                 
293661   0  0.129012  0.003726  0.424278  0.210182  0.097216  0.121474   
         1  0.152782  0.003726  0.447230  0.210182  0.097216  0.156649   
         2  0.152782  0.003533  0.447230  0.210182  0.092662  0.156649   
         3  0.152782  0.003726  0.447230  0.210182  0.097216  0.156649   
         4  0.152782  0.002350  0.447230  0.210182  0.097216  0.156649   

                 f13       f14       f15       f16       f17       f18  \
event_id                                                                 
293661   0  0.034650  0.009060  0.311063  0.051448  0.003730  0.008922   
         1  0.079435  0.009060  0.311063  0.064512  0.003730  0.013719   
         2  0.052644  0.009060  0.311063  0.064512  0.003537  0.013719   
         3  0.079435  0.009060  0.311063  0.064512  0.003730  0.013719   
         4  0.079435  0.009053  0.307945  0.064512  0.002352  0.013719   

                 f19       f20       f21       f22       f23       f24  \
event_id                                                                 
293661   0  0.006000  0.002866  0.011607  0.069193  0.110954  0.013933   
         1  0.004612  0.003109  0.011630  0.067898  0.187072  0.013933   
         2  0.006000  0.003109  0.011630  0.069193  0.187072  0.013933   
         3  0.006000  0.003109  0.011630  0.069193  0.187072  0.013614   
         4  0.006000  0.003109  0.011630  0.069193  0.187072  0.013933   

                 f25       f26       f27       f28       f29       f30  \
event_id                                                                 
293661   0  0.000463  0.000320  0.059297  0.000698  0.013353  0.018471   
         1  0.000000  0.000320  0.059297  0.000656  0.011859  0.042123   
         2  0.000463  0.000320  0.059297  0.000698  0.013353  0.042123   
         3  0.000463  0.000254  0.059297  0.000698  0.013353  0.038616   
         4  0.000463  0.000320  0.059297  0.000698  0.009034  0.042123   

                 f31       f32       f33       f34       f35       f36  \
event_id                                                                 
293661   0  0.079228  0.080813  0.063538  0.002113  0.003758  0.116451   
         1  0.100929  0.080813  0.063538  0.000000  0.003758  0.116451   
         2  0.100929  0.080813  0.063538  0.002113  0.003565  0.116451   
         3  0.100929  0.048994  0.063538  0.002113  0.003758  0.116451   
         4  0.100929  0.065159  0.063538  0.002113  0.002367  0.116451   

                 f37       f38       f39       f40       f41       f42  \
event_id                                                                 
293661   0  0.036222  0.212490  0.003726  0.413493  0.099755  0.186137   
         1  0.036222  0.212490  0.003726  0.497897  0.146564  0.103327   
         2  0.036222  0.212490  0.003533  0.497897  0.146564  0.186137   
         3  0.031307  0.212490  0.003726  0.497897  0.146564  0.176407   
         4  0.035842  0.201068  0.002350  0.497897  0.146564  0.186137   

                 f43       f44       f45       f46       f47       f48  \
event_id                                                                 
293661   0  0.107289  0.011924  0.012905  0.051726  0.037202  0.009559   
         1  0.107289  0.011924  0.012905  0.079162  0.037202  0.009545   
         2  0.107289  0.011924  0.008795  0.079162  0.037202  0.003861   
         3  0.107289  

In [21]:
col_names = ['f%s'%i for i in range(1,58)] + ['M_f%s'%i for i in range(1,58)]

predict_mask.shape[0], df_f.shape

(1742772, (150297, 57))

In [22]:
factors_new = np.zeros((df_f.shape[1], predict_mask.shape[0]))
X = df_f.values
j=0
for i,flag in enumerate(predict_mask):
    if flag:
        factors_new[:,i] = X[j,:]
        j +=1
factors_new.shape, factors.shape

((57, 1742772), (57, 1742772))

In [23]:
factors_new = np.vstack((factors, factors_new))

In [24]:
factors_new.shape

(114, 1742772)

In [25]:
%%time
model_coefs, model_step1prob, model_step2prob, model_likelihood \
= mod.fit_slices(tsav, factors_new, depth=3, lmbd=10, verbose=False, fit_afresh=True)

print "LL new  model"
print model_likelihood
print 
mod.is1 = is1
mod.is2 = is1 
mod.oos = oos

old_coefs, old_step1prob, old_step2prob, old_likelihood \
= mod.fit_slices(tsav, factors, depth=3, lmbd=10, verbose=False, fit_afresh=True)
print "LL old model"
print old_likelihood

. . . . . . . . . . 10
LL new cluster model
[[-1851.31115584 -1995.16006953 -1995.16006953]
 [-1854.66326777 -1991.14291317 -1991.14291317]
 [-1858.29262186 -1988.69182241 -1988.69182241]
 [-1859.38004196 -1984.83079958 -1984.83079958]
 [-1872.69317589 -1972.07729947 -1972.07729947]
 [-1887.93390404 -1962.28002011 -1962.28002011]
 [-1915.06501813 -1947.95300353 -1947.95300353]
 [-1925.7402651  -1939.73386518 -1939.73386518]
 [-1926.96833766 -1939.58826054 -1939.58826054]
 [-1932.65006964 -1934.46768841 -1934.46768841]
 [    0.             0.             0.        ]]

. . . . . . . . . . 10
LL old model
[[-1851.82404552 -1995.80190523 -1995.80190523]
 [-1855.17692183 -1991.83205299 -1991.83205299]
 [-1858.78848725 -1989.37872712 -1989.37872712]
 [-1859.88547983 -1985.49613881 -1985.49613881]
 [-1873.26577532 -1972.64086846 -1972.64086846]
 [-1888.55464921 -1962.77217781 -1962.77217781]
 [-1915.6768923  -1948.29012377 -1948.29012377]
 [-1926.34347024 -1939.98590585 -1939.98590585]
 [-192

##### write the simdata file

In [42]:
def dic_to_tenzor(dic, key, base):
    '''
    tenzor where first dimention is the number of cluster
    0 = no_cluster
    <dic> dictionary cluster's data that to convert in tenzor
    <key> the list of clusters that use 
    <base> no cluster data
    '''
    
    key_0 = dic.keys()[0]
    tenzor = np.zeros((len(key)+1, dic[key_0].shape[0], dic[key_0].shape[1]))
    try:
        tenzor[0,:,:] = base
    except:
        print 'base and dic[k] have the diferent size'
        return
    for i,k in enumerate(key):
        tenzor[i+1,:,:] = dic[k]
    return tenzor

In [43]:
def clusters_number(data, key, av=av):
    """ 
    list with numbers of clusters 
    <data>  pandas Series index = event_id, data = cluster's names
    <key> the list of clusters that use
    """
    
    cl_number = np.zeros((len(av.event_id)))
    for i,k in enumerate(key):
        mask = np.in1d(av.event_id,data.index[data ==k])
        cl_number = np.where(mask,i+1,cl_number)
    return cl_number

In [44]:
def write_simdata(step1probs, oos, coefs, cluster_number=None, file_ = 'simdata.p'):
    '''
    <step1probs> is expected to be a matrix N_slices x len(av). 
    <oos> is a boolean mask denoting the out of sample range. len(oos) shoud equal len(av)
    <coefs> is a coefficient matrix with the size N_slices x 3
    <cluster_number> is an integer array with the cluster numbers per race. Size: len(av)
    '''
    f = file(settings.paths.join(file_), 'wb')
    if cluster_number is None:
        s1p = step1probs[:, oos]
    else:
        cluster_number = cluster_number[oos]
        s1p = step1probs[:, :, oos]
    pickle.dump([s1p, oos, coefs, cluster_number], f)
    f.close()

In [45]:
def write_dic_to_simdata(file_name, old_step1probs, old_coefs, oos, data=None, av =av,
                         cluster_step1probs =None, cluster_coefs =None, cluster_names =None):
    """
    <file_name> is name of file to record
    <old_step1probs> is expected to be a matrix N_slices x len(av)
    <old_coefs> is a coefficient matrix with the size N_slices x 3
    <oos> is a boolean mask denoting the out of sample range. len(oos) shoud equal len(av)
    <data>  pandas Series index = event_id, data = cluster's names
    <cluster_step1probs> is expected to be a dictionary: key is the cluster name and
                        data are the matrix N_slices x len(av) for each cluster
    <cluster_coefs> is a dictionary : key is the cluster name and data and 
                        data are the coefficient matrix with the size N_slices x 3
    <cluster_number> is an integer array with the cluster numbers per race. Size: len(av)
    """
    
    cl_number= np.zeros((len(av.event_id)))
    
    if cluster_names is not None:
        
        s1prob = dic_to_tenzor(cluster_step1probs, cluster_names, old_step1probs)
        coef_s = dic_to_tenzor(cluster_coefs, cluster_names, old_coefs)
        
        for i,k in enumerate(cluster_names):
            mask = np.in1d(av.event_id,data.index[data ==k])
            cl_number = np.where(mask, i+1, cl_number)
        #cl_number = clusters_number(data, cluster_names, av=av)
          
        
    else:
        
        s1prob = np.zeros((1,old_step1probs.shape[0], old_step1probs.shape[1]))
        s1prob[0,:,:] = old_step1probs
        coef_s = np.zeros((1,old_coefs.shape[0], old_coefs.shape[1]))
        coef_s[0,:,:] = old_coefs
        
        

    #write_simdata(s1prob, oos, coef_s, cl_number, file_ = file_name)
    s1prob = s1prob[:,:,oos]
    cl_number = cl_number[oos]
    
    with open (settings.paths.join(file_name), 'wb') as f:
            pickle.dump( [s1prob, oos, coef_s, cl_number], f)
    return